In [ ]:
! pip install pinecone-client==2.2.4

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain, HypotheticalDocumentEmbedder, LLMChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatMlflow

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

from domino_data.vectordb import DominoPineconeConfiguration

from ragatouille import RAGPretrainedModel

import os
import pinecone

In [ ]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/code/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [ ]:
# Initialize Pinecone index
datasource_name = "Rakuten"
conf = DominoPineconeConfiguration(datasource=datasource_name)
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

pinecone.init(
    api_key=api_key,
    environment="domino",
    openapi_config=conf
)

index = pinecone.Index("rakuten")


In [ ]:
chat = ChatMlflow(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="chat-gpt35turbo-sm",
)

conversation_openai = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(llm=chat),
    verbose=False
)

conversation_anthropic = ConversationChain(
        llm=ChatAnthropic(model='claude-2.1'),
        memory=ConversationSummaryMemory(llm=ChatAnthropic(model='claude-2.1')),
        verbose=False
    )

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

In [ ]:
# Setup HyDE

hyde_prompt_template = """You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten
"Please answer the user's question below \n 
Question: {question}
Answer:"""
hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_llm_chain = LLMChain(llm=chat, prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_llm_chain, base_embeddings=embeddings
)

In [ ]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.5

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 5

# Number of texts to return from reranking
NUM_RERANKING_MATCHES = 3

# Create prompt
template = """ You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://www.rakuten.com/help . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""

# Load the reranking model
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Get relevant docs through vector DB
def get_relevant_docs(user_input, num_matches=NUM_TEXT_MATCHES, use_hyde=True):
   
    if use_hyde:
        embedded_query = hyde_embeddings.embed_query(user_input)
    else:
        embedded_query = embeddings.embed_query(user_input)
    
    relevant_docs = index.query(
        vector=embedded_query,
        top_k=num_matches,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input, rerank=True, use_hyde=True):
    
    relevant_docs = get_relevant_docs(user_input)
    
    actual_num_matches = len(relevant_docs["matches"])
    urls = set([relevant_docs["matches"][i]["metadata"]["source"] for i in range(actual_num_matches)])
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    
    if rerank and actual_num_matches >= NUM_RERANKING_MATCHES:
        docs = colbert.rerank(query=user_question, documents=contexts, k=NUM_RERANKING_MATCHES)
        result_indices = [docs[i]["result_index"] for i in range(NUM_RERANKING_MATCHES)]
        contexts = [contexts[index] for index in result_indices]
        urls = [list(urls)[index] for index in result_indices]
    
    # TODO : pull the prompt template from the Hub
    
    # Create prompt
    template = """ You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://www.rakuten.com/help . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""
 
    prompt_template = PromptTemplate(
        input_variables=["url_links", "context"],
        template=template
    )
    
    system_prompt = prompt_template.format( url_links=urls, context=contexts)
 
    return system_prompt
 
# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai", use_hyde=True):

    system_prompt = build_system_prompt(user_input)            
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    if llm_name.lower() == "openai":
        output = conversation_openai.predict(input=messages)
    else:
        output = conversation_anthropic.predict(input=messages)

    return output

In [ ]:
user_question = "text: Google API Disclosure Rakuten uses Google APIs when you use Google to sign in to your Rakuten account. Our use of information from the Google APIs will be in compliance with the Google API Services User Data Policy"

In [ ]:
# Ask a question ; uncomment to test
user_question = input("Please provide your question here :")
result = queryAIModel(user_question)
result